In [1]:
import os
import time
import onnx
import onnxruntime as ort
from utils import *

In [2]:
# Read data
base_data_dir = os.getenv("NBA_DATA_DIR", "nba_data")

X1_train = pd.read_csv(os.path.join(base_data_dir, 'train/X_train_model1.csv'))
X1_train = X1_train.drop('gameId', axis=1)
X1_test = pd.read_csv(os.path.join(base_data_dir, 'test/X_test_model1.csv'))
X1_test = X1_test.drop('gameId', axis=1)

Y1_train = pd.read_csv(os.path.join(base_data_dir, 'train/Y_train_model1.csv'))
Y1_test = pd.read_csv(os.path.join(base_data_dir, 'test/Y_test_model1.csv'))
full1_df = pd.read_csv(os.path.join(base_data_dir, 'train/full_stats.csv'))

X_save_cols = X1_train.columns

# Convert to tensors, pass to dataloader
X1_train = torch.tensor(X1_train.values, dtype=torch.float32)
X1_test = torch.tensor(X1_test.values, dtype=torch.float32)
Y1_train = torch.tensor(Y1_train.values, dtype=torch.float32)
Y1_test = torch.tensor(Y1_test.values, dtype=torch.float32)

train1_data = TensorDataset(X1_train, Y1_train)
test1_data = TensorDataset(X1_test, Y1_test)

train1_loader = DataLoader(train1_data, batch_size=32, shuffle=True)
test1_loader = DataLoader(test1_data, batch_size=32, shuffle=False)

game_ids = full1_df['gameId'].values


In [3]:
model_path = "models/point_diff.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1 = torch.load(model_path, map_location=device)
model1.eval()

onnx_model1_path = "models/model1.onnx"
dummy_input = torch.randn(1, X1_test.shape[1], dtype=torch.float32)

torch.onnx.export(model1, dummy_input, onnx_model1_path,
                  export_params=True, opset_version=20,
                  do_constant_folding=True, input_names=['input'],
                  output_names=['output'], dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}})

print(f"ONNX model saved to {onnx_model1_path}")

onnx_model1 = onnx.load(onnx_model1_path)
onnx.checker.check_model(onnx_model1)

/tmp/ipykernel_2250/4086482454.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model1 = torch.load(model_path, map_location=device)


ONNX model saved to models/model1.onnx


In [5]:
predictions = []
extract_df = full1_df[X_save_cols].values
X_tensor = torch.FloatTensor(extract_df).to(device)
with torch.no_grad():
    # Processsing in batches
    batch_size = 32

    for i in range(0, len(X_tensor), batch_size):
        batch = X_tensor[i:i+batch_size]

        # FF, convert to numpy
        batch_preds = model1(batch)
        predictions.append(batch_preds.cpu().numpy())

# Combine all predictions
all_predictions = np.concatenate(predictions, axis=0)

# Return results
result_df = pd.DataFrame({
    'gameId': game_ids,
    'predicted_point_diff': all_predictions.flatten()
})

In [6]:
X2_train = pd.read_csv(os.path.join(base_data_dir, 'train/X_train_model2.csv'))
X2_test = pd.read_csv(os.path.join(base_data_dir, 'test/X_test_model2.csv'))
Y2_train = pd.read_csv(os.path.join(base_data_dir, 'train/Y_train_model2.csv'))
Y2_test = pd.read_csv(os.path.join(base_data_dir, 'test/Y_test_model2.csv'))

X2_train = X2_train.merge(result_df, on='gameId', how='inner')
X2_train = X2_train.drop('gameId', axis=1)
X2_test = X2_test.merge(result_df, on='gameId', how='inner')
X2_test = X2_test.drop('gameId', axis=1)
full2_df = pd.read_csv(os.path.join(base_data_dir, 'train/full_attendance.csv'))
full2_df = full2_df.merge(result_df, on='gameId', how='inner')

X2_train = torch.tensor(X2_train.values, dtype=torch.float32)
X2_test = torch.tensor(X2_test.values, dtype=torch.float32)
Y2_train = torch.tensor(Y2_train.values, dtype=torch.float32)
Y2_test = torch.tensor(Y2_test.values, dtype=torch.float32)

train2_data = TensorDataset(X2_train, Y2_train)
test2_data = TensorDataset(X2_test, Y2_test)
train2_loader = DataLoader(train2_data, batch_size=32, shuffle=True)
test2_loader = DataLoader(test2_data, batch_size=32, shuffle=False)

In [7]:
model_path = "models/attendance_model.pth"
model2 = torch.load(model_path, map_location=device)
model2.eval()

onnx_model2_path = "models/model2.onnx"
dummy_input = torch.randn(1, X2_test.shape[1], dtype=torch.float32)

torch.onnx.export(model2, dummy_input, onnx_model2_path,
                  export_params=True, opset_version=20,
                  do_constant_folding=True, input_names=['input'],
                  output_names=['output'], dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}})

print(f"ONNX model saved to {onnx_model2_path}")

onnx_model2 = onnx.load(onnx_model2_path)
onnx.checker.check_model(onnx_model2)

ONNX model saved to models/model2.onnx


/tmp/ipykernel_2250/2876132223.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model2 = torch.load(model_path, map_location=device)


In [8]:
model1_size = os.path.getsize(onnx_model1_path)
model2_size = os.path.getsize(onnx_model2_path)
print(f"Model Size on Disk: {model1_size / 1e6:.2f} MB")
print(f"Model Size on Disk: {model2_size / 1e6:.2f} MB")

Model Size on Disk: 3.55 MB
Model Size on Disk: 2.93 MB


In [12]:

ort_session1 = ort.InferenceSession(onnx_model1_path, providers=['CPUExecutionProvider'])
ort_session2 = ort.InferenceSession(onnx_model2_path, providers=['CPUExecutionProvider'])

In [13]:
criterion = nn.MSELoss()

test1_r2, test1_rmse, test1_loss = validate(model1, test1_loader, criterion, device)
print(f"Test R^2: {test1_r2:.4f}, Test RMSE: {test1_rmse:.4f}, Test Model MSE Loss: {test1_loss:.4f}")
test2_r2, test2_rmse, test2_loss = validate(model2, test2_loader, criterion, device)
print(f"Test R^2: {test2_r2:.4f}, Test RMSE: {test2_rmse:.4f}, Test Model MSE Loss: {test2_loss:.4f}")

Test R^2: 0.1917, Test RMSE: 14.2086, Test Model MSE Loss: 201.8852
Test R^2: 0.4043, Test RMSE: 1175.3316, Test Model MSE Loss: 1381404.3058


In [14]:
num_trials = 100
single_sample = X2_test[0].unsqueeze(0).numpy()

# Warm-up
ort_session2.run(None, {ort_session2.get_inputs()[0].name: single_sample})

latencies = []
for _ in range(num_trials):
    start_time = time.time()
    ort_session2.run(None, {ort_session2.get_inputs()[0].name: single_sample})
    latencies.append(time.time() - start_time)


In [15]:
print(f"Inference Latency (median): {np.percentile(latencies, 50) * 1000:.2f} ms")
print(f"Inference Latency (95th percentile): {np.percentile(latencies, 95) * 1000:.2f} ms")
print(f"Inference Latency (99th percentile): {np.percentile(latencies, 99) * 1000:.2f} ms")
print(f"Inference Throughput (single sample): {num_trials / np.sum(latencies):.2f} FPS")


Inference Latency (median): 0.14 ms
Inference Latency (95th percentile): 0.26 ms
Inference Latency (99th percentile): 1.16 ms
Inference Throughput (single sample): 5475.59 FPS


In [16]:
num_batches = 50
batch_input = X2_test[:32].numpy()

# Warm-up
ort_session2.run(None, {ort_session2.get_inputs()[0].name: batch_input})

batch_times = []
for _ in range(num_batches):
    start_time = time.time()
    ort_session2.run(None, {ort_session2.get_inputs()[0].name: batch_input})
    batch_times.append(time.time() - start_time)

batch_fps = (batch_input.shape[0] * num_batches) / np.sum(batch_times)
print(f"Batch Throughput: {batch_fps:.2f} FPS")


Batch Throughput: 47719.14 FPS


In [17]:
print('Model1 Stats: ')
print(f"Model1 Size on Disk: {model1_size/ (1e6) :.2f} MB")
print(f"Test R^2: {test1_r2:.4f}, Test RMSE: {test1_rmse:.4f}, Test Model MSE Loss: {test1_loss:.4f}")
print('\n')
print('Model2 Stats: ')
print(f"Model2 Size on Disk: {model2_size/ (1e6) :.2f} MB")
print(f"Test R^2: {test2_r2:.4f}, Test RMSE: {test2_rmse:.4f}, Test Model MSE Loss: {test2_loss:.4f}")

print(f"Inference Latency (single sample, median): {np.percentile(latencies, 50) * 1000:.2f} ms")
print(f"Inference Latency (single sample, 95th percentile): {np.percentile(latencies, 95) * 1000:.2f} ms")
print(f"Inference Latency (single sample, 99th percentile): {np.percentile(latencies, 99) * 1000:.2f} ms")
print(f"Inference Throughput (single sample): {num_trials/np.sum(latencies):.2f} FPS")
print(f"Batch Throughput: {batch_fps:.2f} FPS")

Model1 Stats: 
Model1 Size on Disk: 3.55 MB
Test R^2: 0.1917, Test RMSE: 14.2086, Test Model MSE Loss: 201.8852


Model2 Stats: 
Model2 Size on Disk: 2.93 MB
Test R^2: 0.4043, Test RMSE: 1175.3316, Test Model MSE Loss: 1381404.3058
Inference Latency (single sample, median): 0.14 ms
Inference Latency (single sample, 95th percentile): 0.26 ms
Inference Latency (single sample, 99th percentile): 1.16 ms
Inference Throughput (single sample): 5475.59 FPS
Batch Throughput: 47719.14 FPS
